In [0]:
from pyspark.sql.functions import date_format, when, col, cast
from funcoes.gerais import *

In [0]:
# Lendo o arquivo origem salvo no volume

df_raw = spark.read.format("csv") \
    .option("header", "true") \
    .option("delimiter", ",") \
    .option("inferSchema", "true") \
    .load("/Volumes/estudo/default/ec/df_t.csv")

In [0]:
# Tratamentos necessários nos dados de entrada:
# time_purchase - voltar a ser apenas HH:mm:ss
# place_origin_return e place_destination_return - tratamento para tirar o valor '0' quando não houver informações de retorno
# fk_return_ota_bus_company - tratamento para tirar o valor '1' quando não houver informações de retorno
# gmv_success - transformação em tipo de dado mais adequado para valor monetário

df_tratado = df_raw.withColumn(
    "time_purchase",
    date_format("time_purchase", "HH:mm:ss")
    ).withColumn("place_origin_return",
                 when(col("place_origin_return") != '0', col("place_origin_return"))
                 .otherwise(None)
    ).withColumn("place_destination_return",
                 when(col("place_destination_return") != '0', col("place_destination_return"))
                 .otherwise(None)
    ).withColumn("fk_return_ota_bus_company",
                 when(col("fk_return_ota_bus_company") != '1', col("fk_return_ota_bus_company"))
                 .otherwise(None)
    ).withColumn("gmv_success",
                 col("gmv_success").cast("decimal(19,2)")
    )

In [0]:
# Escrita na tabela que será usada nos estudos e desenvolvimentos
escrita_tabela(df_tratado, 'estudo.default.df_t')

In [0]:
df_tratado.display()

In [0]:
df_raw.filter(col('nk_ota_localizer_id') == '6d50985c2dc0cd03a1e4907bfd3aa3e29814108be71144662be8bfc9a87d69f9').display()

In [0]:
# Leitura do volume


# Escrita como tabela no Unity Catalog
df.write.mode("overwrite").saveAsTable("meu_catalogo.meu_esquema.minha_tabela")

In [0]:
df_cluster = spark.table("workspace.default.tuser_clustering")

In [0]:
df_origem = spark.table("workspace.default.df_t")

In [0]:
df = df_origem.withColumn("date_purchase_dt", F.col("date_purchase").cast("date")) \
    .withColumn("ano_mes_compra", F.date_format(F.col("date_purchase_dt"), "yyyy-MM")) \
    .withColumn("hora_do_dia_compra", F.hour(F.col("time_purchase")))

In [0]:
dias_semana_num = {
    1: 'Domingo', 2: 'Segunda', 3: 'Terça', 4: 'Quarta',
    5: 'Quinta', 6: 'Sexta', 7: 'Sábado'
}

In [0]:
dias_semana_map_udf = F.udf(lambda d: dias_semana_num.get(d, 'Desconhecido'), StringType())
df = df.withColumn("day_of_week_num", F.dayofweek(F.col("date_purchase_dt"))) # Extrai número do dia da semana
df = df.withColumn("dia_semana_compra", dias_semana_map_udf(F.col("day_of_week_num")))

In [0]:
df = df.withColumn("rota_ida", F.concat_ws(" -> ", F.col("place_origin_departure"), F.col("place_destination_departure")))

In [0]:
df = df.withColumn("rota_volta",
                   F.when(F.col("place_origin_return") != '0',
                        F.concat_ws(" -> ", F.col("place_origin_return"), F.col("place_destination_return")))
                   .otherwise(None))

In [0]:
df = df.withColumn("mes_compra", F.month(F.col("date_purchase_dt"))) \
       .withColumn("dia_do_mes_compra", F.dayofmonth(F.col("date_purchase_dt")))

In [0]:
df = df.withColumn("periodo_do_dia_compra",
                   F.when((F.col("hora_do_dia_compra") >= 0) & (F.col("hora_do_dia_compra") <= 5), "Madrugada")
                   .when((F.col("hora_do_dia_compra") >= 6) & (F.col("hora_do_dia_compra") <= 11), "Manhã")
                   .when((F.col("hora_do_dia_compra") >= 12) & (F.col("hora_do_dia_compra") <= 17), "Tarde")
                   .otherwise("Noite"))

In [0]:
meses_map = {
    1: 'Janeiro', 2: 'Fevereiro', 3: 'Março', 4: 'Abril',
    5: 'Maio', 6: 'Junho', 7: 'Julho', 8: 'Agosto',
    9: 'Setembro', 10: 'Outubro', 11: 'Novembro', 12: 'Dezembro'
}

In [0]:
map_mes_udf = F.udf(lambda m: meses_map.get(m, 'Não definido'), StringType())

In [0]:
grouped_metrics_basic = df.groupBy("fk_contact").agg(
    F.count(F.col("nk_ota_localizer_id")).alias("total_compras"),
    F.avg(F.col("gmv_success")).alias("valor_medio_compra"),
    F.sum(F.col("total_tickets_quantity_success")).alias("total_tickets_compras"),
    F.sum(F.when(F.col("rota_volta").isNotNull(), 1).otherwise(0)).alias("total_compras_com_volta")
)

In [0]:
grouped_metrics_basic.display()

In [0]:
def calculate_mode_spark(dataframe, group_cols, target_col, output_col_name):
    window_spec_count = Window.partitionBy(*group_cols, target_col)

    df_counted = dataframe.withColumn(
        "count_target",
        F.count(target_col).over(window_spec_count)
    )

    window_spec_rank = Window.partitionBy(*group_cols).orderBy(F.col("count_target").desc(), F.col(target_col))

    df_mode = df_counted.withColumn(
        "rank_mode",
        F.row_number().over(window_spec_rank)
    ).filter(F.col("rank_mode") == 1) \
     .select(*group_cols, F.col(target_col).alias(output_col_name))

    return df_mode

In [0]:
rota_ida_frequente = calculate_mode_spark(
    df, ["fk_contact"], "rota_ida", "local_partida_ida_local_destino_ida_frequente"
)

In [0]:
rota_volta_frequente = calculate_mode_spark(
    df.filter(F.col("rota_volta").isNotNull()),
    ["fk_contact"], "rota_volta", "local_partida_volta_local_destino_volta_frequente"
)

In [0]:
dia_preferido = calculate_mode_spark(
    df, ["fk_contact"], "dia_semana_compra", "dia_da_semana_preferido_de_compra"
)

In [0]:
periodo_dia_preferido = calculate_mode_spark(
    df, ["fk_contact"], "periodo_do_dia_compra", "periodo_do_dia_preferido_para_compra"
)

In [0]:
epoca_ano_preferida = calculate_mode_spark(
    df, ["fk_contact"], "mes_compra", "epoca_do_ano_preferido_da_compra"
)

In [0]:
epoca_mes_preferida = calculate_mode_spark(
    df, ["fk_contact"], "dia_do_mes_compra", "epoca_do_mes_preferido_da_compra"
)

In [0]:
final_metrics = grouped_metrics_basic \
    .join(rota_ida_frequente, on=["fk_contact"], how="left") \
    .join(rota_volta_frequente, on=["fk_contact"], how="left") \
    .join(dia_preferido, on=["fk_contact"], how="left") \
    .join(periodo_dia_preferido, on=["fk_contact"], how="left") \
    .join(epoca_ano_preferida, on=["fk_contact"], how="left") \
    .join(epoca_mes_preferida, on=["fk_contact"], how="left")

In [0]:
final_metrics = final_metrics.withColumn(
    "epoca_do_ano_preferido_da_compra",
    map_mes_udf(F.col("epoca_do_ano_preferido_da_compra"))
)

In [0]:
final_metrics.filter(F.col('fk_contact') == 'cbae228abf4b9f465f42d8d6f97a83f2b76c077aea36c3b7303915796cf3dac3').display()

In [0]:
df_join = final_metrics \
    .join(df_cluster, on=["fk_contact"], how="left")

In [0]:
df_final = df_join.withColumn("media_tickets_por_compra", F.round(F.col("total_tickets_compras") / F.col("total_compras"), 2)) \
    .withColumn("porcentagem_viagens_com_volta", F.round(F.col("total_compras_com_volta") / F.col("total_compras") * 100, 2)) \
    .select(
        "fk_contact",
        F.col("cluster").alias("cd_cluster"),
        F.col("segmento_do_cliente").alias("ds_cluster"),
        "recencia_dias",
        F.round("valor_total_gasto", 2).alias("valor_total_gasto"),
        F.round("valor_medio_compra", 2).alias("valor_medio_compra"),
        F.col("frequencia_de_compras").alias("quantidade_compras"),
        "total_tickets_compras",
        "media_tickets_por_compra",
        "total_compras_com_volta",
        "porcentagem_viagens_com_volta",
        F.col("local_partida_ida_local_destino_ida_frequente").alias("rota_ida_frequente"),
        F.col("local_partida_volta_local_destino_volta_frequente").alias("rota_volta_frequente"),
        F.col("dia_da_semana_preferido_de_compra").alias("dia_semana_preferido_compra"),
        F.col("epoca_do_mes_preferido_da_compra").alias("dia_mes_preferido_compra"),
        F.col("periodo_do_dia_preferido_para_compra").alias("periodo_dia_preferido_compra"),
        F.col("epoca_do_ano_preferido_da_compra").alias("mes_preferido_compra")
    )

In [0]:
df_final.write.mode("overwrite").saveAsTable("workspace.default.tperfil_cliente")

In [0]:
df_final.filter(F.col('total_compras_com_volta') > 1).display()

In [0]:
df_final.filter(F.col('fk_contact') == 'cbae228abf4b9f465f42d8d6f97a83f2b76c077aea36c3b7303915796cf3dac3').display()

In [0]:
%sql
CREATE TABLE T_HIST_PERFIL_CLIENTE (
        fk_contact                STRING      PRIMARY KEY,
        ds_cluster                VARCHAR2(30)    NOT NULL,
    qt_total_compras          NUMBER(7)       NOT NULL,
    vl_medio_compras          NUMBER(10,2)    NOT NULL,
    qt_total_passagens        NUMBER(5)       NOT NULL,
    qt_viagens_retorno        NUMBER(5)       NOT NULL,
    ds_rota_frequente         VARCHAR2(100)   NOT NULL,
    hr_media_compra           VARCHAR2(8)     NOT NULL,
    ds_dia_semana_preferido   VARCHAR2(15)    NOT NULL,
    ds_periodo_dia_preferido  VARCHAR2(15)    NOT NULL,
    ds_epoca_mes_compra       VARCHAR2(20)    NOT NULL,
    dt_referencia_perfil      DATE            NOT NULL
);
